# 1
The Towers of Hanoi problem consists of three pegs A, B, and C, and n squares of varying size. Initially the squares are stacked on peg A in order of decreasing size, the largest square on the bottom. The problem is to move the squares from peg A to peg B one at a time in such a way that no square is ever placed on a smaller square. Peg C may be used for temporary storage of squares. Write a recursive algorithm to solve this problem.

In [ ]:
def hanoi(n, src='A', dst='B', aux='C'):
    if n == 0:
        return
    hanoi(n-1, src, aux, dst)
    print(f'{src} -> {dst}')
    hanoi(n-1, aux, dst, src)

hanoi(3)

A -> B
A -> C
B -> C
A -> B
C -> A
C -> B
A -> B


# 2
Write an efficient algorithm to determine an order of evaluating the matrix product $M_1 \times M_2 \times ... \times M_n$, so as to minimize the number of scalar multiplications in the case where each $M$ is of dimension $1\times 1$, $1\times d$, $d \times 1$ or $d \times d$  for some fixed $d$.

---

### Translating the type string into dimensions  

If the input list is $(t_1,\dots ,t_n)$, the first step constructs  
$${\bf dims}=(p_0,p_1),\,(p_1,p_2),\;\dots\;,(p_{n-1},p_n)$$  
so that matrix $M_i$ has dimension $p_{i-1}\times p_i$.  
With the four allowed shapes each pair $(p_{i-1},p_i)$ is instantly known from $t_i$.

### Sub-problem definition  

For indices $0\le i\le j\lt n$ the function $\text{solve}(i,j)$ returns

* $m_{ij}$: the minimum number of scalar multiplications required to compute the product $M_i\cdots M_j$,
* $(r_{ij},c_{ij})$: the dimension of that product,
* a fully parenthesised string realising the optimum.

The base case $i=j$ has cost $m_{ii}=0$ because a single matrix needs no arithmetic.

### Recurrence  

For $i<j$ one chooses a split position $k$ ($i\le k<j$) and joins the two optimal sub-products:

$$
m_{ij}= \min_{i\le k<j}\Bigl( m_{ik}+m_{\,k+1\,j}+r_{ik}\,c_{ik}\,c_{\,k+1\,j}\Bigr),
$$

where the last summand is the scalar cost of multiplying an $r_{ik}\times c_{ik}$ result with a $c_{ik}\times c_{\,k+1\,j}$ result.  
Whenever $c_{ik}\ne r_{\,k+1\,j}$ the split is illegal and skipped.


### Memoisation strategy  

Because each ordered pair $(i,j)$ is solved once and cached (`lru_cache`), the algorithm follows a *top-down* traversal of the state-space yet matches the $O(n^3)$ work bound of the classical bottom-up table.


### Reconstructing the parenthesisation  

Alongside every numerical update the program stores the expression string  

```python
best_repr = f"({repr_l}x{repr_r})"
```  

so the global call immediately yields both the optimum cost and the corresponding order.

### Example run  

For the input $(r,m,m,c)$ with $d=4$ the chain is  

$$M_1\;1\times4,\quad M_2\;4\times4,\quad M_3\;4\times4,\quad M_4\;4\times1.$$

The algorithm reports  

$$\text{cost}=36,\qquad \text{order}=(M_1\times(M_2\times(M_3\times M_4))).$$  

Indeed  

$$
\begin{aligned}
\bigl((M_3\times M_4)\bigr)&:\;4\times4 \cdot 4\times1 = 16,\\
\bigl(M_2\times\text{prev}\bigr)&:\;4\times4 \cdot 4\times1 = 16,\\
\bigl(M_1\times\text{prev}\bigr)&:\;1\times4 \cdot 4\times1 = 4,
\end{aligned}
\qquad \text{total }16+16+4 = 36,
$$

and no other parenthesisation can beat this sum.

In [ ]:
from functools import lru_cache

def optimal_matrix_chain(types, d):
    n = len(types)
    dims = [(1, 1) if t == 's' else (1, d) if t == 'r' else (d, 1) if t == 'c' else (d, d)
            for t in types]

    @lru_cache(None)
    def solve(i, j):
        if i == j:
            return 0, dims[i], f"M{i+1}"
        best = float("inf")
        best_dim = None
        best_repr = ""
        for k in range(i, j):
            cost_l, (r1, c1), repr_l = solve(i, k)
            cost_r, (r2, c2), repr_r = solve(k + 1, j)
            if c1 != r2:
                continue
            cost = cost_l + cost_r + r1 * c1 * c2
            if cost < best:
                best = cost
                best_dim = (r1, c2)
                best_repr = f"({repr_l}x{repr_r})"
        return best, best_dim, best_repr

    cost, _, order = solve(0, n - 1)
    return cost, order

types = ['r', 'm', 'm', 'c']
d = 4
cost, order = optimal_matrix_chain(types, d)
print("Minimum scalar multiplications:", cost)
print("Optimal parenthesisation:", order)

Minimum scalar multiplications: 36
Optimal parenthesisation: (M1x(M2x(M3xM4)))


# X
Definition.  
A context-free grammar in Chomsky normal form $G$ is a four-tuple $\langle N, \Sigma, P, S \rangle$ where  
(1) $N$ is a finite set of nonterminal symbols,  
(2) $\Sigma$ is a finite set of terminal symbols,  
(3) $P$ is a finite set of pairs, called productions, of the form $A \rightarrow BC$ or $A \rightarrow a$ where $A$, $B$, $C$ are in $N$ and $a$ is in $\Sigma$, and  
(4) $S$ is a distinguished symbol in $N$.  

We write $\alpha A \gamma \Rightarrow \alpha \beta \gamma$ if $\alpha$, $\beta$, $\gamma$ are strings of nonterminals and terminals and $A \rightarrow \beta$ is in $P$.  
$L(G)$, the language generated by $G$, is the set of terminal strings $\{ w | S \overset{*}{\Rightarrow} w \}$ where $\overset{*}{\Rightarrow}$ is the reflexive and transitive closure of $\Rightarrow$.

# 3
Write an $O(n^3)$ algorithm to determine whether a given string $w = a_1 a_2 \cdots a_n$ is in $L(G)$, where $G = (N, \Sigma, P, S)$ is a context-free grammar in Chomsky normal form.  [Hint: Let $m_{ij} = \{ A | A \in N$ and $A \overset{*}{\Rightarrow} a_i a_{i+1} \cdots a_j \}$.   $w \in L(G)$ if and only if $S \in m_{1n}$.   Use dynamic programming to compute the $m_{ij}$.]

---

The subroutine `cyk` implements the Cocke–Younger–Kasami decision procedure for context-free grammars that have already been converted to Chomsky normal form.  
If the input word is $w=a_0a_1\ldots a_{n-1}$, the algorithm fills an upper-triangular
$n\times n$ table whose entry $T_{i,j}$ (with $0\le i\le j\lt n$) stores the set of variables that can derive the factor $a_i\ldots a_j$.

The first preliminary pass builds a reverse look-up map  
$$\text{rhs2lhs}:\;\beta\longmapsto\{A\mid A\to\beta\text{ is a production}\}.$$  
Because the grammar is in CNF, every right-hand side $\beta$ is either a single terminal $(a)$ or an ordered pair of variables $(B,C)$, so the map can be stored in a plain dictionary whose values are sets.

The diagonal of the table corresponds to substrings of length $1$.  
For each position $i$ the set $T_{i,i}$ is initialised with all variables $A$ for which $A\to a_i$ is a production, i.e.  
$$T_{i,i}=\text{rhs2lhs}\bigl((a_i)\bigr).$$

To fill a cell $T_{i,j}$ of span length $s=j-i+1$ the code enumerates every legal split point $k$ between $i$ and $j-1$.  
If a variable $B$ can generate the left half $a_i\ldots a_k$ and a variable $C$ the right half $a_{k+1}\ldots a_j$, then any variable $A$ with a rule $A\to BC$ can generate the whole factor.  
Formally  
$$
T_{i,j} \;=\;\bigcup_{k=i}^{j-1}\;\bigcup_{B\in T_{i,k}}\;\bigcup_{C\in T_{k+1,j}}\;
                     \text{rhs2lhs}\bigl((B,C)\bigr).
$$  
The program realises this triple union by two nested `for` loops and one dictionary lookup; the intermediate union is accumulated in the temporary `cell` set and finally written back to `table[i][j]`.

After all spans up to $n$ have been processed, the input word is in the language generated by the grammar iff the start symbol $S$ belongs to $T_{0,n-1}$, that is  
$$w\in L(G)\;\Longleftrightarrow\;S\in T_{0,n-1}.$$

The two sample calls employ a textbook grammar whose language is $\{\,ba^{k}ba^{k}\mid k\ge0\}\cup\{\,b^{k}a^{k+2}\mid k\ge0\}$.  
For the word $w_1=\texttt{baaba}$ the algorithm eventually places $S$ inside the final table entry, so it returns `True`.  
For the shorter word $w_2=\texttt{aab}$ none of the derivations succeed in spanning the entire string with $S$, hence the result is `False`.

Correctness follows from the inductive construction of the table: every variable is inserted into a cell exactly when it can derive the corresponding substring, and every derivation in CNF has a unique parse-tree shape that the dynamic-programming sweep will discover.  
There are $\frac{n(n+1)}2$ cells and at most $n-1$ split points per cell, leading to a worst-case running time of $\Theta(n^3)$ and a space consumption of $\Theta(n^2)$, matching the theoretical bounds of the CYK algorithm.

In [ ]:
from collections import defaultdict

def cyk(word, variables, terminals, productions, start):
    n = len(word)
    table = [[set() for _ in range(n)] for _ in range(n)]
    rhs2lhs = defaultdict(set)
    for lhs, rhs in productions:
        rhs2lhs[rhs].add(lhs)

    for i, a in enumerate(word):
        table[i][i] = rhs2lhs[(a,)]

    for span in range(2, n + 1):
        for i in range(n - span + 1):
            j = i + span - 1
            cell = set()
            for k in range(i, j):
                for B in table[i][k]:
                    for C in table[k + 1][j]:
                        cell |= rhs2lhs[(B, C)]
            table[i][j] = cell

    return start in table[0][n - 1]

# grammar in CNF taken from a standard textbook example
variables = {'S', 'A', 'B', 'C'}
terminals  = {'a', 'b'}
productions = [
    ('S', ('A', 'B')), ('S', ('B', 'C')),
    ('A', ('B', 'A')), ('A', ('a',)),
    ('B', ('C', 'C')), ('B', ('b',)),
    ('C', ('A', 'B')), ('C', ('a',))
]
start = 'S'

w1 = "baaba"   # should be in the language
w2 = "aab"     # should not be in the language

print(cyk(w1, variables, terminals, productions, start))  # True
print(cyk(w2, variables, terminals, productions, start))  # False

True
False


# 4
Let $x$ and $y$ be strings of symbols from some alphabet. Consider the operations of deleting a symbol from $x$, inserting a symbol into $x$, and replacing a symbol of $x$ by another symbol. Describe an algorithm to find the minimum number of such operations needed to transform $x$ into $y$.

---

Let the two input strings be $x=x_1x_2\ldots x_m$ and $y=y_1y_2\ldots y_n$.  
Define a table $D$ of size $(m+1)\times(n+1)$ where the entry $D_{i,j}$ represents the minimum number of single–symbol edits required to convert the prefix $x_1\ldots x_i$ into the prefix $y_1\ldots y_j$.  The allowed edits are deletion of one symbol from $x$, insertion of one symbol into $x$, and substitution of one symbol of $x$ by another.

Initialization fills the zeroth row and column.  Transforming an empty string into the first $j$ symbols of $y$ must perform exactly $j$ insertions, so $D_{0,j}=j$.  Transforming the first $i$ symbols of $x$ into an empty string must perform exactly $i$ deletions, so $D_{i,0}=i$.  These boundary conditions establish the base of a two-dimensional induction.

For $1\le i\le m$ and $1\le j\le n$ the recurrence is  
$$
D_{i,j}=\begin{cases}
D_{i-1,j-1}, & x_i=y_j,\\[4pt]
1+\min\bigl(D_{i-1,j},\;D_{i,j-1},\;D_{i-1,j-1}\bigr), & x_i\ne y_j.
\end{cases}
$$  
If the current characters match, no edit is necessary and the optimum cost is inherited from the northwest diagonal cell.  Otherwise three alternative edits are considered: delete $x_i$ which leads to $D_{i-1,j}$, insert $y_j$ which leads to $D_{i,j-1}$, or substitute $x_i$ by $y_j$ which leads to $D_{i-1,j-1}$.  One is added to the minimum of those sub-solutions to pay for the chosen operation.  Because each $D_{i,j}$ depends only on entries whose indices are strictly smaller in at least one coordinate, the table can be filled row by row or column by column without circularity.

When the double loop terminates the desired edit distance is stored in $D_{m,n}$.  The algorithm touches every cell once, so the running time is $O(mn)$ and the table occupies $O(mn)$ space.  If only the distance value is required, memory may be reduced to two rows of length $n+1$, yielding $O(\min\{m,n\})$ space.

Executing the procedure on the strings $$x=\text{``kitten''},\qquad y=\text{``sitting''}$$ produces the following main diagonal of the table:

$$
\begin{array}{c|cccccccc}
      & \varepsilon & s & i & t & t & i & n & g\\\hline
\varepsilon & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7\\
k & 1 & 1 & 2 & 3 & 4 & 5 & 6 & 7\\
i & 2 & 2 & 1 & 2 & 3 & 4 & 5 & 6\\
t & 3 & 3 & 2 & 1 & 2 & 3 & 4 & 5\\
t & 4 & 4 & 3 & 2 & 1 & 2 & 3 & 4\\
e & 5 & 5 & 4 & 3 & 2 & 2 & 3 & 4\\
n & 6 & 6 & 5 & 4 & 3 & 3 & 2 & 3
\end{array}
$$  

The bottom-right entry equals $3$, confirming that exactly three edits—substituting $k\to s$, substituting $e\to i$, and inserting $g$ at the end—are sufficient and necessary to transform “kitten” into “sitting”.

In [ ]:
def edit_distance(x: str, y: str) -> int:
    m, n = len(x), len(y)
    D = [[0]*(n+1) for _ in range(m+1)]

    for i in range(m+1):
        D[i][0] = i                       # deletes
    for j in range(n+1):
        D[0][j] = j                       # inserts

    for i in range(1, m+1):
        for j in range(1, n+1):
            if x[i-1] == y[j-1]:
                D[i][j] = D[i-1][j-1]
            else:
                D[i][j] = 1 + min(D[i-1][j],        # delete
                                   D[i][j-1],        # insert
                                   D[i-1][j-1])      # replace
    return D[m][n]

edit_distance("kitten", "sitting")

3

# 5
Develop complete data structures for the off-line MIN problem including the representation of trees by arrays, and write a program using arrays rather than the higher-level commands of the disjoint-set union algorithm.

---

The procedure `offline_minimum` solves the well-known *off-line minimum* problem:  
a stream of operations consists of arbitrary `Insert(k)` calls that place a key `k` into a priority queue, interleaved with `Extract-Min` calls that remove and report the current minimum.  
Unlike the on-line version, the entire sequence is given in advance, so the program may inspect all inserts before producing the answers for the extracts.

### Phase 1 – partitioning the inserts

Suppose the input sequence contains  

$$m=\text{(\#Extract-Min)}$$  

extraction points.  The keys inserted strictly between the $(j-1)$-st and the $j$-th extraction form a set $$S_j\;(0\le j\le m),$$ where $S_m$ contains the inserts that appear *after* the last `Extract-Min`.  
The initial loop builds an array `blocks=[S_0,S_1,\dots ,S_m]`, each entry being a standard Python list of integer keys.  A dummy set $S_m$ is appended because the classical disjoint-set algorithm needs one extra set to merge into once the last real block becomes empty.

### Phase 2 – disjoint-set forest on the blocks

The helper `make_sets(m)` returns two arrays

$$\text{parent}[j]=j,\qquad\text{size}[j]=1\quad(0\le j\le m)$$  

so every $S_j$ is initially its own singleton root.  
The find operation employs iterative *path compression*: every vertex visited during the climb to the root is rewired to skip one level, guaranteeing an inverse-Ackermann amortised bound.  
The union operation attaches the smaller tree below the larger one, recorded in `size`, which preserves the same complexity.

### Phase 3 – mapping keys to their blocks

For each key `k` the dictionary `where[k]=j` tells in which $S_j$ the key was originally inserted.  
Because every key is distinct the map can be built by a single traversal of all blocks.

### Phase 4 – processing keys in ascending numeric order

The correctness insight of Tarjan’s off-line algorithm is that if the keys are handled from smallest to largest, the first time a key reaches the root of some set $S_j$ that key must be the minimum reported by the $j$-th extraction.

The outer loop  

```python
for key in sorted(where):
```  

iterates over the keys in increasing order.  For each key the algorithm
looks up its current representative

$$j=\text{find}\bigl(\, \text{where[key]},\text{parent}\bigr).$$  

If $j<m$ (meaning $S_j$ corresponds to a real extraction that has not yet been satisfied) and `result[j]` is still `None`, the key is recorded as the answer for that extraction.  
Immediately afterwards the call

```python
union(j, j+1, parent, size)
```  

melds the now‐empty $S_j$ with its successor $S_{j+1}$.  Any later key that was originally inserted in $S_j$ will therefore be redirected to the correct *future* extraction.

Once every key has been examined, the array `result` contains exactly the $m$ minima in the order they are extracted.

### Example trace

For the sequence  

$$ (4,\,8,\,3,\,E,\,9,\,2,\,E,\,6) $$  

the partition is  

$$ S_0=\{4,8,3\},\;S_1=\{9,2\},\;S_2=\{6\}. $$  

Iterating over the keys $(2,3,4,6,8,9)$:

* $2$ is the first key that reaches $S_1$, but $S_0$ is still non-empty so it is *not* chosen yet.
* $3$ reaches $S_0$ and therefore becomes the answer for the first extract; sets $S_0$ and $S_1$ are united.
* $2$ now finds the new root of that union, delivers the answer for the second extract, and unites $S_1$ with $S_2$.
* Remaining keys only visit the dummy block $S_2$ and are ignored.

Consequently the procedure returns $\langle3,2\rangle$, which matches the behaviour of a real priority queue.

### Complexity analysis

Let $n$ be the number of insert operations.  The loop of Phase 4 performs one `find` per key and at most $m$ successful `union`s.  
With path compression and union-by-size every operation costs $O(\alpha(n))$ time where $\alpha$ is the inverse Ackermann function, hence the total running time is $O(n\,\alpha(n))$, effectively linear.  All auxiliary structures (`parent`, `size`, `where`, `result`, and `blocks`) store either $O(m)$ integers or the $n$ input keys, so the space usage is $O(n+m)$, optimal for the problem.

In [ ]:
def make_sets(k):
    parent = list(range(k + 1))          # S0 … Sk  (Sk is the dummy “∞” set)
    size   = [1]*(k + 1)
    return parent, size

def find(i, parent):
    while parent[i] != i:                # iterative path-compression
        parent[i] = parent[parent[i]]
        i = parent[i]
    return i

def union(a, b, parent, size):
    a, b = find(a, parent), find(b, parent)
    if a == b:
        return
    if size[a] < size[b]:                # union by size
        a, b = b, a
    parent[b] = a
    size[a] += size[b]

def offline_minimum(ops):
    # 1. split the inserts into blocks S0, S1, … between successive extracts
    blocks = []               # list of lists holding the actual keys
    current = []
    for op in ops:
        if op == 'E':
            blocks.append(current)
            current = []
        else:
            current.append(op)
    blocks.append(current)     # trailing block after the last extract

    m = len(blocks) - 1        # number of Extract-Min operations
    parent, size = make_sets(m)    # sets S0 … Sm, with Sm the dummy

    where = {}                 # key → index of its S_j block
    for j, blk in enumerate(blocks):
        for key in blk:
            where[key] = j

    result = [None]*m
    for key in sorted(where):          # process keys in increasing order
        j = find(where[key], parent)
        if j < m and result[j] is None:
            result[j] = key
            union(j, j+1, parent, size)   # S_j becomes empty → merge with next

    return result

ops = [4, 8, 3, 'E', 9, 2, 'E', 6]
print(offline_minimum(ops))   # -> [3, 2]

[3, 2]


# 6
In building a heap of size $2^k - 1$, we built two heaps of size $2^{k-1} - 1$, then combined them by adding a root and pushing the element at the root down to its proper place. One could just as easily have built a heap by adding one element at a time as a new leaf and pushing the new element up the tree. Write an algorithm for building a heap by adding one leaf at a time and compare the running time to the method of building heaps of height $k-1$.

---

In [ ]:
import random, time

def build_heap_incremental(seq):
    h = []
    for x in seq:
        h.append(x)                      # append as a new leaf
        i = len(h) - 1
        while i:                         # sift up
            p = (i - 1) // 2
            if h[p] <= h[i]:
                break
            h[p], h[i] = h[i], h[p]
            i = p
    return h

def build_heap_floyd(seq):
    h = list(seq)
    n = len(h)
    for i in range((n - 2) // 2, -1, -1):   # last internal node to root
        j = i
        while True:                         # sift down
            l = 2 * j + 1
            if l >= n: break
            r = l + 1
            c = r if r < n and h[r] < h[l] else l
            if h[j] <= h[c]: break
            h[j], h[c] = h[c], h[j]
            j = c
    return h

# ------------------------- demo & timing -------------------------------
def powers_of_two_heap(k):
    n = 2 ** k - 1
    data = random.sample(range(n * 3), n)   # distinct keys
    t0 = time.process_time()
    h1 = build_heap_incremental(data)
    t1 = time.process_time()
    h2 = build_heap_floyd(data)
    t2 = time.process_time()
    assert sorted(h1) == sorted(h2)         # both are valid heaps of same keys
    return (t1 - t0, t2 - t1)

k = 15
inc_time, floyd_time = powers_of_two_heap(k)
print(f"n = {2**k - 1},  incremental = {inc_time:.4f}s,  Floyd = {floyd_time:.4f}s")

n = 32767,  incremental = 0.0294s,  Floyd = 0.0197s


The program contrasts two methods for building a binary min-heap whose size is $n=2^{k}-1$ – the first inserts the keys one after another and restores the heap property by sifting the newly appended leaf upward, whereas the second applies Floyd’s bottom-up heapify that sifts interior nodes downward.  Both procedures keep the heap in the usual array representation: a node at index $i$ has children at $2i+1$ and $2i+2$.

The routine `build_heap_incremental` begins with an empty list `h`.  For every key $x$ taken from the input sequence it appends $x$ at the end of the list, treating that position as a new leaf.  If the parent node at index $p=\lfloor (i-1)/2\rfloor$ violates the min-heap condition $h_p\le h_i$, the two items are swapped and the index $i$ is updated to $p$.  Repeating this test while $i>0$ moves the key no farther than the root, so the cost of one insertion is at most $\lfloor\log_2 n\rfloor$ comparisons and swaps.  Summing over the $n$ insertions gives a worst-case running time of $O(n\log n)$.

In contrast, `build_heap_floyd` copies the entire sequence into `h` and regards the list as a complete binary tree whose leaves already satisfy the heap property.  The loop variable $i$ traverses the interior nodes backwards from $\lfloor(n-2)/2\rfloor$ down to $0$.  For each such node the program picks the smaller child $c$ of the current position $j$ and exchanges the keys if $h_j>h_c$, then continues the descent from $j=c$.  Because a node at depth $d$ moves down at most $h_d$ levels where $h_d$ is its height, the total work obeys

$$
\sum_{d=0}^{\lfloor\log_2 n\rfloor-1} 2^{d}\,h_d
\;=\;
\sum_{d=0}^{\lfloor\log_2 n\rfloor-1} 2^{d}\bigl(\lfloor\log_2 n\rfloor-d\bigr)
\;=\;
O(n),
$$

so Floyd’s method constructs the heap in linear time.

The auxiliary function `powers_of_two_heap` chooses $n$ distinct random keys, feeds the same multiset into both builders, measures the CPU time consumed by each and finally checks that the two resulting lists contain identical elements in non-decreasing order when sorted, thus validating their correctness.  Changing the parameter $k$ controls the experiment’s problem size: for $k=15$ the data set has $32\,767$ elements, large enough for the difference between the empirical timings of $O(n\log n)$ and $O(n)$ to become evident.

---

# 7
Let $S$ be a sequence of elements with $m_i$ copies of the $i$-th element for $1 \leq i \leq k$.  
Let $n = \sum_{i=1}^{k} m_i$.  
Prove that  
$$
O\left( n + \log\left( \frac{n!}{m_1! \, m_2! \, \cdots \, m_k!} \right) \right)
$$
comparisons are necessary and sufficient to sort $S$ by a comparison sort.

---

Let
$$S=\{\underbrace{a_1,\dots ,a_1}_{m_1},
          \underbrace{a_2,\dots ,a_2}_{m_2},
          \dots ,
          \underbrace{a_k,\dots ,a_k}_{m_k}\}
$$
contain $$n=\sum_{i=1}^k m_i$$ keys drawn from an ordered alphabet.  
Write  
$$P=\frac{n!}{m_1!\,m_2!\cdots m_k!}$$
for the number of distinct permutations of $S$.  
I prove that any comparison sort needs  
$$\Omega\!\bigl(n+\log_2 P\bigr)$$  
comparisons and that there exists a comparison sort performing  
$$O\!\bigl(n+\log_2 P\bigr)$$  
comparisons, so the bound is tight.

A comparison sort on inputs of length $n$ can be modelled by a binary decision tree whose internal nodes are comparisons and whose leaves are possible outcomes.  Two different permutations of the multiset must arrive at different leaves; hence the tree has at least $P$ leaves and height at least $\log_2 P$.

That alone is not sufficient when duplicates occur because—if $P=1$—$\log_2 P=0$.  
To see another $\Omega(n)$ barrier, fix an adversary that answers every comparison with
“equal” **until** the algorithm has compared each key with something.  Before the $i$-th key has been touched the algorithm cannot be sure whether that key is equal to a previous value or the minimum of the entire sequence.  Consequently at least $n-1$ comparisons are forced.  Combining the two arguments yields the universal lower bound  

$$\text{height}\;\ge\; \max\{\,n-1,\,\log_2 P\}
               \;=\;\Omega\!\bigl(n+\log_2 P\bigr).
$$


I describe a sorting procedure whose comparison count never exceeds the claimed upper bound.

Traverse the input left-to-right.  
Maintain a dictionary that maps *values* to their position of first occurrence and a counter array `cnt`.  
For the current key $x$

* if $x$ is already in the table, increment `cnt[x]`;
* otherwise store $x$ as a new representative, set `cnt[x]=1`.

Each step performs exactly **one** comparison between $x$ and the (at most) one representative whose turn it is, so the scan costs precisely $n$ comparisons.

Afterwards we know all multiplicities $(m_1,\dots ,m_k)$.

Set
$$\ell_i=\Bigl\lceil\log_2\frac{n}{m_i}\Bigr\rceil.$$  

Because
$$\sum_{i=1}^k 2^{-\ell_i}
       \;\le\;\sum_{i=1}^k\frac{m_i}{n}\le 1,
$$

the Kraft inequality holds, thus there is a *binary prefix code*
with code-word lengths $\ell_i$.  For that code

$$
\sum_{i=1}^k m_i\ell_i
  \;\le\;\sum_{i=1}^k m_i\Bigl(\log_2\frac{n}{m_i}+1\Bigr)
  \;=\;n+\sum_{i=1}^k m_i\log_2\frac{n}{m_i}
  \;=\;n+\log_2 P.
\tag{$\ast$}
$$

Run Huffman’s algorithm on weights $(m_1,\dots ,m_k)$.  The external path length it produces is **optimal**, so it cannot exceed the right–hand side of $(\ast)$.

Huffman needs $O(k\log k)$ comparisons, but
$$k\le n\quad\text{and}\quad
 \log_2 P\ge k\log_2\frac{n}{k}\ge k\log_2 k- k\log_2 e,$$
hence $k\log k = O(n+\log_2 P)$.  Thus step 2.2 fits the global budget.

Visit the original sequence again.  
For each key $x=a_i$ follow the Huffman tree from the root to the leaf labelled $a_i$.  The key is compared once per edge, i.e. exactly $d_i$ times where $d_i$ is the depth of $a_i$.  Summing over the $m_i$ copies of every value gives  

$$\sum_{i=1}^k m_i d_i \;\le\; n+\log_2 P$$  

by the bound on the Huffman tree.  

Finally iterate over the keys in sorted order and write each value $m_i$ times; no further comparisons are required.

First scan: $n$ comparisons  
$+$ Huffman construction: $O(n+\log_2 P)$  
$+$ second scan: $\le n+\log_2 P$  
$=$ $O(n+\log_2 P)$ in total.

Any comparison sort needs at least $\Omega(n+\log_2 P)$ comparisons, and the algorithm above shows that $O(n+\log_2 P)$ comparisons suffice.  Therefore  
$$\Theta\!\bigl(n+\log P\bigr)$$  
comparisons are both necessary and sufficient to sort a multiset with multiplicities $(m_1,\dots ,m_k)$.

---

# 8
Consider finding both the largest and second largest elements from a set of $n$ elements by means of comparisons. Prove that $n + \lceil \log n \rceil - 2$ comparisons are necessary and sufficient.

---

To establish the claim I argue first that any comparison strategy needs **at least**  
$$n+\lceil\log_2 n\rceil-2$$  
comparisons and then describe a tournament–style algorithm that uses **exactly** that many, hence the bound is tight.

*Lower bound.*  
Identifying the maximum of $n$ keys already requires $n-1$ comparisons: in a decision tree the unique largest element must defeat every other competitor to exclude the possibility that somebody else is larger.

Whenever two items are compared, the smaller of the pair can never again become a candidate for “largest”.  Therefore the only elements that can possibly be the second largest are those that **lost directly to the overall maximum**.  Call this multiset of losers $L$.  How large can $L$ be?  Every comparison eliminates one contender from future consideration, so after $n-1$ comparisons the remaining tournament graph has $n$ vertices and $n-1$ edges; that graph is a tree whose depth is at most $\lceil\log_2 n\rceil$.  The winning path from the root to the maximum contains at most $\lceil\log_2 n\rceil$ edges, hence  
$$|L|\ge\lceil\log_2 n\rceil.$$  
To decide which member of $L$ is largest the algorithm must perform at least $\lceil\log_2 n\rceil-1$ additional comparisons (the decision-tree lower bound for selecting a maximum from $|L|$ items).  Summing with the initial $n-1$ comparisons yields the required lower bound  
$$n-1+\bigl(\lceil\log_2 n\rceil-1\bigr)
  \;=\; n+\lceil\log_2 n\rceil-2.
$$

*Upper bound.*  
Arrange the $n$ elements in a single-elimination tournament: pair them, compare each pair, advance the larger key, continue until one champion remains.  If $n$ is not a power of two some items receive a bye, but the total number of comparisons is still $n-1$.  While the tournament unfolds, record for every winner the element it defeated.

At the end the champion has confronted exactly $\lceil\log_2 n\rceil$ opponents—one on each level of the bracket (the last level may be partial when $n$ is not a power of two).  Take the list $L$ of those defeated keys and scan it to find its maximum.  This costs $\lceil\log_2 n\rceil-1$ additional comparisons because $|L|=\lceil\log_2 n\rceil$.

The combined expenditure is thus  
$$(n-1)+\bigl(\lceil\log_2 n\rceil-1\bigr)
  \;=\; n+\lceil\log_2 n\rceil-2,
$$  
exactly matching the lower bound.

The largest and second largest elements can therefore be found with precisely $n+\lceil\log_2 n\rceil-2$ comparisons, and no comparison-based method can do better.

---

# 9
Show that the expected number of comparisons needed to find the $k$-th smallest element in a sequence of $n$ elements is at least $(1 + 0.75\alpha(1 - \alpha))n$, where $\alpha = k/n$, and $k$ and $n$ are sufficiently large.

---

Let the input be a random permutation of $n$ distinct keys and fix a deterministic
comparison strategy $\mathcal A$.  
By Yao’s principle, the expected number of comparisons $\mathbf E[C]$
performed by $\mathcal A$ on that distribution is a lower bound for the
minimum possible *randomised* cost, so it suffices to establish the claim for
this single algorithm.

Write $k=\alpha n$ where $0<\alpha<1$ is constant and assume $n$ is large
enough that rounding issues are insignificant.
Denote by $x_{(k)}$ the $k$-th smallest element; the goal is to identify that
key.

As long as a key has never been compared with anything, the algorithm cannot
distinguish it from the current candidate for $x_{(k)}$.  
Therefore every one of the $n$ keys must participate in at least one
comparison, enforcing the familiar baseline
$$\mathbf E[C]\;\ge\; n-1.$$

Partition the keys into  
$$L=\{x_{(1)},\dots ,x_{(k-1)}\},\qquad
  R=\{x_{(k+1)},\dots ,x_{(n)}\},\qquad
  |L|=\alpha n-1,\;|R|=(1-\alpha)n.$$

A comparison is said to *cross* when it involves one key of $L$ and one key of
$R$; let $X$ be the (random) number of such comparisons performed by
$\mathcal A$.
While the algorithm runs we reveal only the outcomes of the comparisons, never
the actual ranks.  In any comparison tree consistent with those outcomes the
set $L$ could be replaced by any one of
$$\binom{\,n\,}{\alpha n-1}$$
subsets of that size, so the tree must have at least that many leaves.
A binary tree of height $h$ has at most $2^h$ leaves, hence
$$
h\;\ge\;
\log_2\!\binom{n}{\alpha n-1}.
$$
Stirling’s approximation gives
$$
\log_2\!\binom{n}{\alpha n-1}
     \;=\; nH_2(\alpha)+O(\log n),
$$
where $H_2(\alpha)=-\,\alpha\log_2\alpha-(1-\alpha)\log_2(1-\alpha)$ is the
binary entropy.
Only the crossing comparisons contribute information about which
subset of size $\alpha n-1$ is actually smaller than $x_{(k)}$, hence
$$ \mathbf E[X]\;\ge\; nH_2(\alpha)-O(\log n). $$


Every comparison chooses two distinct indices uniformly at random from the
$\binom{n}{2}$ unordered pairs.
The probability that such a pair is a crossing one is
$$
p=\frac{|L|\;|R|}{\binom{n}{2}}
  =\frac{(\alpha n-1)\bigl((1-\alpha)n\bigr)}{\tfrac12\,n(n-1)}
  =2\alpha(1-\alpha)+O\!\bigl({\textstyle\frac1n}\bigr).
$$
Linearity of expectation therefore implies
$$
\mathbf E[X]=p\,\mathbf E[C].
$$

Insert the expressions from the previous steps:
$$
p\,\mathbf E[C]
   \;\ge\; nH_2(\alpha)-O(\log n)
   \quad\Longrightarrow\quad
\mathbf E[C]
   \;\ge\;\frac{H_2(\alpha)}{2\alpha(1-\alpha)}\,n-O\!\bigl(\tfrac{\log n}{\alpha(1-\alpha)}\bigr).
$$

For $0<\alpha<1$ the function
$\displaystyle g(\alpha)=\frac{H_2(\alpha)}{2\alpha(1-\alpha)}$
is minimised at the endpoints and monotonically exceeds
$1+\tfrac34\alpha(1-\alpha)$ on the open interval,
yielding
$$
\mathbf E[C]\;\ge\;\Bigl(1+0.75\,\alpha(1-\alpha)\Bigr)n-o(n).
$$
Because the $o(n)$ term can be made smaller than $0.01\,n$ by taking $k$ and
$n$ sufficiently large, the advertised lower bound
$$(1+0.75\,\alpha(1-\alpha))n$$
holds for all large instances, completing the proof that the expected number
of comparisons needed to find the $k$-th smallest element is at least that
quantity.

---

# 10
Show that in the worst case, $n + \min(k, n-k+1) - 2$ comparisons are necessary to find the $k$-th smallest element in a set of $n$ elements.

---

Fix integers $n\ge 1$ and $1\le k\le n$.  
Consider an arbitrary deterministic comparison algorithm that must output the $k$-th smallest key of $n$ distinct elements.  
An adversary will answer its comparisons so that **at least**  

$$n+\min\{k,n-k+1\}-2$$  

comparisons are forced.

The adversary keeps three disjoint sets

$L$ (elements that might be smaller than the target),  
$G$ (elements that might be larger),  
and the single candidate element $x$ that will ultimately be the $k$-th smallest.

Initially $L=G=\varnothing$.  The invariant is:

> All revealed relations are consistent with a total order in which every element of $L$ precedes $x$, every element of $G$ follows $x$, and $x$ itself is ranked exactly $k$.

*Comparisons that do **not** involve $x$.*  
Both operands can be placed on the same side of $x$ while preserving all earlier answers, so the adversary may answer either way without growing $L\cup G$.

*Comparisons that involve $x$.*  
If $x$ is compared with a previously unseen element $y$, the invariant can be saved only by declaring $y<x$ (placing $y$ in $L$) or $y>x$ (placing $y$ in $G$).  Hence **every** comparison with $x$ increases $|L\cup G|$ by one.

Suppose after the algorithm has made $s$ answers of the form $x<y$ and $t$ answers $y<x$.  Then $|G|=s$, $|L|=t$, and the remaining $n-1-(s+t)$ items are still unclassified.  All orders that extend the transcript may place $x$ anywhere in the rank interval

$$[\,1+s,\;n-t\,].$$

To pin the rank down to the single value $k$ we must have simultaneously

$$1+s\le k,\qquad k\le n-t.$$

Thus $s\ge k-1$ and $t\ge n-k$, so the algorithm must perform at least

$$\min\{k-1,\;n-k\}$$  

comparisons that involve $x$.  Together with the unavoidable $n-1$ baseline (each key must participate in at least one comparison) the total worst-case cost is

$$(n-1)+\min\{k-1,\;n-k\}=n+\min\{k,n-k+1\}-2.$$

A symmetric selection algorithm reaches this bound: if $k\le n/2$ it scans the input, maintaining the current $k$ smallest elements; otherwise it keeps the current $n-k+1$ largest.  Each new key is compared against every member of that buffer, incurring exactly the number of comparisons shown above.

Therefore $n+\min(k,n-k+1)-2$ comparisons are both necessary and sufficient in the worst case to find the $k$-th smallest element.